# テキスト分類モデル構築

## 環境構築

### googleドライブをマウントする

In [0]:
# Google Drive のマウント
from google.colab import drive
drive.mount('/content/drive')

## データ準備

|説明変数|目的変数|
|:-:|:-:|
|診断結果のテキスト|12星座|

### pandasでデータを読み込む

In [0]:
# データフレームを操作するためのライブラリ
import pandas as pd

# CSVファイルを読み込んだあと、dfにデータをデータフレームに格納する
df = pd.read_csv('/content/drive/My Drive/pj_horoscopenlp/dataset/shiitake_textdata.csv')

# 読み込んだデータの先頭行を表示する
df.head(5)

# データクレンジング

### Mecabのインストール

In [0]:
# ColabでMecabをインストールする
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

### ライブラリのインポート

- どんどん追加していくスタイル

## 分かち書き

### 「分かち書き」することによって形態素に分解する

In [0]:
#　形態素解析に必要なライブラリをインポートする
  import MeCab

  # 文字列をtextに代入
  text = '「無理にまとめようとしなくて良いかも」の茶色が出ています。今週の牡牛座はちょっと不思議な時間を過ごしていて、「細かく、やることが多い」というときです。「気づいたら手をつける」という、そういう所用が多くあります。何かをやっている最中に「あ、こっちもやらなきゃ」とか「こっちは変更になった」とか、そういう細かい軌道修正も必要になったりします。こういうときは落ち着いて、あまり「ひとつのゴール」を目指さなくていいです。「目の前にあるものをひとつひとつ終わらせていくこと。別に今の私はまとまりがなくても構わない。手が空いたら気晴らしをしにいく時間も持つ」みたいな感じで大丈夫です。'

  # taggerオブジェクトを使う準備
  tagger = MeCab.Tagger("-Owakati")

  # 分かち書きの結果をresultに代入
  result = tagger.parse(text)

  # 結果表示
  print(result)

## 形態素解析

In [0]:
# ライブラリのインポート
import MeCab

# 文字列をtextに代入
text = ('「無理にまとめようとしなくて良いかも」の茶色が出ています。今週の牡牛座はちょっと不思議な時間を過ごしていて、「細かく、やることが多い」というときです。「気づいたら手をつける」という、そういう所用が多くあります。何かをやっている最中に「あ、こっちもやらなきゃ」とか「こっちは変更になった」とか、そういう細かい軌道修正も必要になったりします。こういうときは落ち着いて、あまり「ひとつのゴール」を目指さなくていいです。「目の前にあるものをひとつひとつ終わらせていくこと。別に今の私はまとまりがなくても構わない。手が空いたら気晴らしをしにいく時間も持つ」みたいな感じで大丈夫です。')

# taggerオブジェクトを使う準備
tagger = MeCab.Tagger()

# 分かち書きの結果をresultに代入
result = tagger.parse(text)

# 結果表示
print(result)

### 新しい辞書をインストールする

In [0]:
# 形態素解析に必要な形態素解析解析エンジンのインポート
import MeCab

In [0]:
# mecab-ipadic-NEologdのダウンロード
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n

### 辞書を使って、文字列に対して品詞を与える

In [0]:
# ライブラリのインポート
import MeCab

# 文字列をtextに代入
text = ('あなたは今、色々な展望が開けていくチャンスを目の前に')

# taggerオブジェクトを使う準備
tagger = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

# 分かち書きの結果をresultに代入
result = tagger.parse(text)

# 結果表示
print(result)

## ストップワード除去（名詞、動詞、形容詞以外の形態素を取り除く）

In [0]:
  # ストップワードの除去：使用頻度の高い言葉を処理対象外にする
  import MeCab
  
  # mecab-ipadic-NEologd辞書指定してオブジェクト生成
  tagger = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd")
  tagger.parse("")
  
  # 形態素解析の結果をリストで取得、単語ごとにリストの要素に入ってる
  node = tagger.parseToNode("「無理にまとめようとしなくて良いかも」の茶色が出ています。今週の牡牛座はちょっと不思議な時間を過ごしていて、「細かく、やることが多い」というときです。「気づいたら手をつける」という、そういう所用が多くあります。何かをやっている最中に「あ、こっちもやらなきゃ」とか「こっちは変更になった」とか、そういう細かい軌道修正も必要になったりします。こういうときは落ち着いて、あまり「ひとつのゴール」を目指さなくていいです。「目の前にあるものをひとつひとつ終わらせていくこと。別に今の私はまとまりがなくても構わない。手が空いたら気晴らしをしにいく時間も持つ」みたいな感じで大丈夫です。")
  result = []

  #助詞や助動詞は拾わない
  while node is not None:
      # 品詞情報取得
      # Node.featureのフォーマット：品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用形,活用型,原形,読み,発音
      hinshi = node.feature.split(",")[0]
      if hinshi in ["名詞"]:
          # 表層形の取得、単語の文字が入ってる
          result.append(node.surface)
      elif hinshi in["動詞","形容詞"]:
          # 形態素情報から原形情報取得
          result.append(node.feature.split(",")[6])
      node = node.next

  print(result)
 

# データセット作成 - 単語リストをつくる

## データ整形用のメソッドを作る

### 意味が推測できる品詞だけを抽出するメソッド

In [0]:
# textという引数を使うtokenizeという名前のメソッドを作成
def tokenize(text):
    # 形態素解析を行って、nodeという変数に格納
    node = mecab.parseToNode(text)
    # while 文は指定した条件式が真の間、処理を繰り返し実行し続ける
    # つまり、テキストデータの中で、名詞と分類される単語がなくなるまで実行し続ける
    while node:
        # もし、nodeが持っているfeature[0]列にある情報が名詞である場合
        if node.feature.split(',')[0] == '名詞':
            # 一旦、処理を止めて大文字を小文字に変換する
            yield node.surface.lower()
        # 一つのnodeに対して処理が終わったので、次のnodeの処理を実行する
        node = node.next

In [0]:
# textという引数を使うtokenizeという名前のメソッドを作成
def tokenize2(text):
    # 形態素解析を行って、nodeという変数に格納
    node = mecab.parseToNode(text)
    # while 文は指定した条件式が真の間、処理を繰り返し実行し続ける
    # つまり、テキストデータの中で、名詞と分類される単語がなくなるまで実行し続ける
    while node:
      if node.feature.split(",")[0] == u"名詞" or node.feature.split(',')[0] == '動詞':
        node = node.next
        # 一旦、処理を止めて大文字を小文字に変換する
            #yield node.surface.lower()
      # 一つのnodeに対して処理が終わったので、次のnodeの処理を実行する

In [0]:
# textという引数を使うtokenizeという名前のメソッドを作成
def tokenize3(text):
    # 形態素解析を行って、nodeという変数に格納
    node = mecab.parseToNode(text)
    # while 文は指定した条件式が真の間、処理を繰り返し実行し続ける
    # つまり、テキストデータの中で、名詞と分類される単語がなくなるまで実行し続ける
    while node:
      if node.feature.split(",")[0] == u"名詞" or node.feature.split(',')[0] == '形容詞':
        node = node.next

### テキストデータを一つずつ、形態素解析して返すメソッド

In [0]:
def get_words_main(content):
    '''
    テキストデータを一つずつ、形態素解析して返す
    '''
    return [token for token in tokenize3(content)]

### 全てのテキストデータを形態素解析してリストにして返すメソッド





In [0]:
def get_words(contents):
    '''
    辞書型になっているデータに対して、形態素解析してリストにして返すメソッド
    '''
    # retという空の配列を作成
    ret = []
    # for文=繰り返し処理がしたい　items()は辞書型のデータに対してk, content各要素のキーkeyと値valueの両方に対してforループ処理
    for k, content in contents.items():
      # append()はリストに対して要素を追加するというメソッド
      # retというリストに対して、contentから形態素解析した結果を追加する
      ret.append(get_words_main(content))
    return ret

### 形態素解析する必要があるデータをtext_dataという変数に格納する

In [0]:
# textdataだけを配列に代入

text_data = df["text"]
print(text_data)

### 記事を一つずつ形態素解析する

In [0]:
  import MeCab
  mecab = MeCab.Tagger('mecabrc')


# 記事を一つずつ形態素解析する
# wordという変数にtext_dataにある全て
words = get_words(text_data)
print(words)

## Gensimを使って特徴語辞書をつくる



In [0]:
from gensim import corpora

# words には、形態素解析した単語リストが入っている
#dictionaryメソッドを使って、単語に対して、IDを振る
dictionary = corpora.Dictionary(words)

print(dictionary)

### IDと単語の対応表作成（重要度可視化用）

In [0]:
d = dictionary.token2id

word_to_id = pd.DataFrame(d.items(), columns=['word', 'id'])
word_to_id = word_to_id[['id', 'word']]
word_to_id.head()

## 特徴ベクトルへの変換


 **dictionary.filter_extremes** 

- no_below : 単語が使われている文章の数が設定値未満の時、その単語を削除
- no_above : 単語が使われている文章の割合が設定値以上のとき、その単語を削除
- keen_n : 上記二つの設定にかかわらず、指定した数の単語を保持
- keep_tokens : 指定した単語を保持

### ベクトル化

In [0]:
from gensim import corpora

dictionary = corpora.Dictionary(words)

# 「出現頻度が20未満の単語」と「30%以上の文書で出現する単語」を排除
dictionary.filter_extremes(no_below = 20, no_above = 0.3)


#dictionary.save_as_text("./tmp/dictionary.txt") で、作成した辞書を保存可能
#dictionary = corpora.Dictionary.load_from_text("./tmp/dictionary.txt") で読み込み

# bowは今回のスクリプトでは未使用
#bow = [dictionary.doc2bow(word) for word in words]

### 特徴ベクトルへの変換

- vec2dense：複数のリストに対して一括でベクトル変換できるようにするためのモジュール化する
-0 return以降がBoW表現の文章に対する、特徴ベクトルへの変換処理

In [0]:
from gensim import matutils

def vec2dense(vec, num_terms):
  # corpus2denseの引数は１文書単位なので、以下のようにモジュール化してリスト内包表記すると良い。
  # matutils.corpus2denseは、Gensimコーパスを密行列に変換するメソッド
  # num_termsはコーパス内の用語数、今回はコーパスの用語数は同じだからイコール
  return list(matutils.corpus2dense([vec], num_terms=num_terms).T[0]) 

In [0]:

# doc2bow()はbag-of-words形式にテキストを変換してくれるメソッド
# 特徴語の数（len(dictionary)）だけdoc2bowでベクトル変換した行列を表示する
# wordsという抽出した特徴語の数(len(words))の回数だけ（range()）繰り返して、iに代入
data_all = [vec2dense(dictionary.doc2bow(words[i]), len(dictionary)) for i in range(len(words))]

In [0]:
# この形が特徴ベクトル
print(data_all[0])

In [0]:
len(data_all[0])

## 機械学習

### 学習用データセット（目的変数の数値化）

In [0]:
# 正解ラベル

label = df["label"]

### 説明変数

In [0]:
# 説明変数
train_data = data_all

## データ分割

In [0]:
from sklearn.model_selection import train_test_split

#パラメーターの過剰適合を防ぐために訓練セットとテストセットを分割しておく
X_train, X_test, Y_train, Y_test = train_test_split(
    train_data, label, test_size=0.2, random_state=0)

調べるとどうやら、学習データは、1行に一つのデータがあるような構成でなければならないらしい。


### 決定木

In [0]:
# 決定木に必要なライブラリを読み込み
from sklearn.tree import DecisionTreeClassifier

# 分類器の呼び出し 
dt = DecisionTreeClassifier(max_depth=3, random_state=0) #今回木の深さは3に設定する
# 学習
dt.fit(X_train, Y_train)

In [0]:
# 予測
dt_pred = dt.predict(X_test)
dt_pred[0:20]

In [0]:
# 検証データで推論した結果をdt_predへ代入
dt_pred = dt.predict(X_test)

# 実際の正解データと照合して正解率を算出
accuracy_score(Y_test, dt_pred)

### ランダムフォレスト

In [0]:
# ランダムフォレストに必要なライブラリを読み込み
from sklearn.ensemble import RandomForestClassifier

# 分類器の呼び出し 
rf = RandomForestClassifier(random_state=0)

# 学習
rf.fit(X_train, Y_train)

In [0]:
# 検証データで推論した結果をdt_predへ代入
rf_pred = rf.predict(X_test)

# 実際の正解データと照合して正解率を算出
accuracy_score(Y_test, rf_pred)

In [0]:
# 予測
rf_pred = rf.predict(X_test)
rf_pred[0:20]

### ニューラルネットワーク

In [0]:
# Neural Networkに必要なライブラリのインストール
from sklearn.neural_network import MLPClassifier

# 分類器の呼び出し 
NN = MLPClassifier(random_state=0)

# 学習
NN.fit(X_train, Y_train)

In [0]:
# 検証データで推論した結果をdt_predへ代入
NN_pred = NN.predict(X_test)

# 実際の正解データと照合して正解率を算出
accuracy_score(Y_test, NN_pred)

In [0]:
# 予測
NN_pred = NN.predict(X_test)
NN_pred[0:20]

## 精度検証

In [0]:
# モデルの評価に使用するライブラリを読み込む
from sklearn import metrics

# 正解率を表示
result = pd.DataFrame(
    [round(metrics.accuracy_score(Y_test, rf_pred) * 100, 2),
     round(metrics.accuracy_score(Y_test, ｄｔ_pred) * 100, 2),
     round(metrics.accuracy_score(Y_test, NN_pred) * 100, 2)],
    index=['ランダムフォレスト','決定木', 'Neural Network'],
).sort_values(by=0, ascending=False)
result.columns = ['精度：Accuracy']

result

# 未知のデータで検証

In [0]:
import pandas as pd

# データフレームを操作するためのライブラリ
import pandas as pd

# CSVファイルを読み込んだあと、dfにデータをデータフレームに格納する
test_df = pd.read_csv('/content/drive/My Drive/pj_horoscopenlp/dataset/test_data_sep.csv')

# 読み込んだデータの先頭行を表示する
test_df.head(5)

# 参考サイト

## データ取得元

- [しいたけ占い - 星座・今週の運勢 | 占い | VOGUE GIRL](https://voguegirl.jp/horoscope/shiitake/)


# ハマったところ

- Runtime Error
  - -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd
  - -d /var/lib/mecab/dic/mecab-ipadic-neologd
  
  辞書のバージョンが異なる場合はうまく行かない
  ローカルだと下の辞書でも上手く言ったが、Colabだと上じゃないとうまく実行されなかった